In [1]:
import os
import numpy as np
import pandas as pd
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp

from lvmdrp.functions import imageMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}
SLITMAP = Table(drp.fibermap.data)

In [2]:
frames_table = md.get_metadata(tileid="*", mjd=60177)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]

[INFO]: loading metadata store at /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5
[INFO]: found 513 frames in store '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5'
[INFO]: number of frames after filtering 513
[INFO]: total number of frames found 513


In [3]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
os.makedirs(masters_path, exist_ok=True)

In [4]:
arc_analogs = frames_table.query("imagetyp=='arc'").groupby(["camera"])

marc_paths = []
for camera in arc_analogs.groups:
    arcs = arc_analogs.get_group(camera)
    arc = arcs.iloc[0].to_dict()
    
    lamp = MASTER_ARC_LAMPS[camera[0]]
    # if camera[0] == "b":
    #     arcs = arcs.query("hgne & not neon & not argon & not xenon")
    # else:
    #     arcs = arcs.query("neon & not hgne & not argon & not xenon")

    print(arcs.to_string())
    
    # define master paths for target frames
    mpixmask_path = os.path.join(masters_path, f"lvm-mpixmask-{camera}.fits")
    mbias_path = os.path.join(masters_path, f"lvm-mbias-{camera}.fits")
    mdark_path = os.path.join(masters_path, f"lvm-mdark-{camera}.fits")
    
    # define master frame path
    marc_path = path.full("lvm_master", drpver=drpver, tileid=arc["tileid"], mjd=masters_mjd, kind=f"marc_{lamp}", camera=arc["camera"])
    marc_paths.append(marc_path)
    os.makedirs(os.path.dirname(marc_path), exist_ok=True)
    
    # preprocess and detrend arcs
    darc_paths = []
    for arc in arcs.to_dict("records"):
        arc_path = path.full("lvm_raw", camspec=arc["camera"], **arc)
        parc_path = path.full("lvm_anc", drpver=drpver, kind="p", imagetype=arc["imagetyp"], **arc)
        darc_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=arc["imagetyp"], **arc)
        darc_paths.append(darc_path)
        
        imageMethod.preproc_raw_frame(in_image=arc_path, out_image=parc_path, in_mask=mpixmask_path)
        imageMethod.detrend_frame(in_image=parc_path, out_image=darc_path, in_bias=mbias_path, in_dark=mdark_path, in_slitmap=SLITMAP)

[INFO]: starting preprocessing of raw image 'sdR-s-b1-00003453.fits.gz'


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
53     s    1111  60177  60177      arc  sp1     b1    3453     10.0  False  False    False  False   True  False   False  excellent     0      1       0        0  sdR-s-b1-00003453.fits
59     s    1111  60177  60177      arc  sp1     b1    3463    150.0  False   True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003463.fits
68     s    1111  60177  60177      arc  sp1     b1    3455     50.0  False  False    False  False   True  False   False  excellent     0      1       0        0  sdR-s-b1-00003455.fits
104    s    1111  60177  60177      arc  sp1     b1    3461     10.0  False   True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003461.fits
135    s    1111  60177  60177      arc  sp1     b1    3454     10.0  

[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.71, 2.71, 2.68, 2.69] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 974.00 +/- 1.08 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 981.00 +/- 1.11 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 982.00 +/- 1.28 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 982.00 +/- 1.28 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b1.fits
[INFO]: building pixel mask
[INFO]: 106029 (0.64 %) pixels masked
[INFO]: replacing 106029 masked pixels wit

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
0      s    1111  60177  60177      arc  sp2     b2    3455     50.0  False  False    False  False   True  False   False  excellent     0      1       0        0  sdR-s-b2-00003455.fits
28     s    1111  60177  60177      arc  sp2     b2    3464    150.0  False   True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003464.fits
63     s    1111  60177  60177      arc  sp2     b2    3461     10.0  False   True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003461.fits
81     s    1111  60177  60177      arc  sp2     b2    3454     10.0  False  False    False  False   True  False   False  excellent     0      1       0        0  sdR-s-b2-00003454.fits
126    s    1111  60177  60177      arc  sp2     b2    3468     60.0  

[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 50.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.45, 2.46, 2.46, 2.5] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1003.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1000.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1007.00 +/- 1.26 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1003.00 +/- 1.27 (ADU)
[INFO]: using header RDNOISE = [3.0, 3.0, 2.8, 2.8] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-b2.fits
[INFO]: building pixel mask
[INFO]: 168734 (1 %) pixels masked
[INFO]: replacing 168734 masked pixels wit

...

[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.76, 2.65, 2.89, 2.78] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 980.00 +/- 1.08 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 979.00 +/- 1.08 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 989.00 +/- 1.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 990.00 +/- 1.18 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z1.fits
[INFO]: building pixel mask
[INFO]: 157770 (0.95 %) pixels masked
[INFO]: replacing 157770 masked pixels wit

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
89     s    1111  60177  60177      arc  sp2     z2    3461     10.0  False   True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003461.fits
100    s    1111  60177  60177      arc  sp2     z2    3458     10.0   True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003458.fits
116    s    1111  60177  60177      arc  sp2     z2    3454     10.0  False  False    False  False   True  False   False  excellent     0      1       0        0  sdR-s-z2-00003454.fits
119    s    1111  60177  60177      arc  sp2     z2    3465    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003465.fits
136    s    1111  60177  60177      arc  sp2     z2    3463    150.0  

[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 10.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 988.00 +/- 1.13 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 982.00 +/- 1.18 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 998.00 +/- 1.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 998.00 +/- 1.19 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z2.fits
[INFO]: building pixel mask
[INFO]: 155551 (0.93 %) pixels masked
[INFO]: replacing 155551 masked pixels with

    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
19     s    1111  60177  60177      arc  sp3     z3    3456     50.0  False  False    False  False   True  False   False  excellent     0      1       0        0  sdR-s-z3-00003456.fits
40     s    1111  60177  60177      arc  sp3     z3    3468     60.0   True   True    False  False   True  False   False  excellent     0      1       0        0  sdR-s-z3-00003468.fits
50     s    1111  60177  60177      arc  sp3     z3    3461     10.0  False   True    False  False  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003461.fits
85     s    1111  60177  60177      arc  sp3     z3    3454     10.0  False  False    False  False   True  False   False  excellent     0      1       0        0  sdR-s-z3-00003454.fits
97     s    1111  60177  60177      arc  sp3     z3    3463    150.0  

[INFO]: starting preprocessing of raw image 'sdR-s-z3-00003456.fits.gz'
[INFO]: using header IMAGETYP = 'arc'
[INFO]: exposure time 50.0 (s)
[INFO]: using header TRIMSEC = ['[1:2043, 2041:4080]', '[2078:4120, 2041:4080]', '[1:2043, 1:2040]', '[2078:4120, 1:2040]']
[INFO]: using header BIASSEC = ['[2044:2060, 2041:4080]', '[2061:2077, 2041:4080]', '[2044:2060, 1:2040]', '[2061:2077, 1:2040]']
[INFO]: using header GAIN = [2.6, 2.65, 2.57, 2.72] (e-/ADU)
[INFO]: median and standard deviation in OS quadrant 1: 1009.00 +/- 1.18 (ADU)
[INFO]: median and standard deviation in OS quadrant 2: 1012.00 +/- 1.14 (ADU)
[INFO]: median and standard deviation in OS quadrant 3: 1012.00 +/- 1.28 (ADU)
[INFO]: median and standard deviation in OS quadrant 4: 1010.00 +/- 1.29 (ADU)
[INFO]: using header RDNOISE = [3.6, 3.6, 3.4, 3.6] (e-)
[INFO]: flipping along X-axis
[INFO]: updating header with per quadrant stats
[INFO]: loading master pixel mask from lvm-mpixmask-z3.fits
[INFO]: building pixel mask
[INFO